In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
deso = gpd.read_file('output/enriched_data.gpkg')
deso["x"] = deso.centroid.map(lambda p: p.x)
deso["y"] = deso.centroid.map(lambda p: p.y)

In [ ]:
X = deso.loc[:, deso.columns.isin(['deso', 'kommun', 'lan', 'kommunnamn', 'lannamn', 'geometry', 'index_right', 'Lkfv',
       'Vdnamn', 'Distriktkod', 'Distriktnamn', 'S', 'SD', 'M', 'V', 'C', 'KD',
       'MP', 'L', 'STÖRSTA_PARTI']) == False]
#y = deso.loc[:, ['S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L', 'STÖRSTA_PARTI']]
y = deso['STÖRSTA_PARTI']
random_seed = 5
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random_seed, test_size = 0.2)

In [ ]:
#param_grid = {
#    "scaler__with_mean": [True, False],
#    "scaler__with_std": [True, False],
#    "pca__n_components": [0.5, 0.75, 0.95],
#    "pca__whiten": [True, False],
#    "logistic__C": np.logspace(-4, 4, 4),
#}
param_grid = {
    "pca__n_components": [0.5, 0.75, 0.95],
    "logistic__C": np.logspace(-4, 4, 4),
}
scaler = StandardScaler()
pca = PCA()
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("logistic", logistic)])
search = GridSearchCV(pipe, param_grid, n_jobs=2)
search.fit(X_train, y_train.values)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)